# * VINSIGHT : Gross Adds
    Prepaid(GEO Channel) : DTAC รอพี่หมอ re-run ต้องไม่นับ Wholesales, AE

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000101CS' --Prepaid Gross Adds
        , 'TB1S000101CS' --Prepaid Gross Adds : TMH
        , 'DB1S000101CS' --Prepaid Gross Adds : DTAC
        , 'B1S000101CG' --Prepaid Gross Adds - GEO Channel
        , 'TB1S000101CG' --Prepaid Gross Adds : TMH - GEO Channel
        , 'DB1S000101CG' --Prepaid Gross Adds : DTAC - GEO Channel
        
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB2S010100CS' --Postpaid Gross Adds B2C : TMH
        , 'DB2S010100CS' --Postpaid Gross Adds B2C : DTAC
        , 'B2S020100CS' --Postpaid Gross Adds B2B
        , 'TB2S020100CS' --Postpaid Gross Adds B2B : TMH
        , 'DB2S020100CS' --Postpaid Gross Adds B2B : DTAC
        , 'B2S010100CG' --Postpaid Gross Adds B2C - GEO Channel
        , 'TB2S010100CG' --Postpaid Gross Adds B2C : TMH - GEO Channel
        , 'DB2S010100CG' --Postpaid Gross Adds B2C : DTAC - GEO Channel
        , 'B2S020100CG' --Postpaid Gross Adds B2B - GEO Channel
        , 'TB2S020100CG' --Postpaid Gross Adds B2B : TMH - GEO Channel
        , 'DB2S020100CG' --Postpaid Gross Adds B2B : DTAC - GEO Channel
        
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB3S000102CG' --TOL Gross Adds Connected : Consumer - GEO Channel
        , 'TB3S000102D1CS' --TOL Gross Adds Connected : Consumer (Install Location)
        , 'TB3S000102D1CG' --TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
        , 'TB3S000103CS' --TOL Gross Adds Connected : DataService
        , 'TB3S000103CG' --TOL Gross Adds Connected : DataService - GEO Channel
        , 'TB3S000103D1CS' --TOL Gross Adds Connected : DataService (Install Location)
        , 'TB3S000103D1CG' --TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
        , 'TB3S000104CS' --TOL Gross Adds Registration : Consumer
        , 'TB3S000104CG' --TOL Gross Adds Registration : Consumer - GEO Channel
        , 'TB3S000105CS' --TOL Gross Adds Registration : DataService
        , 'TB3S000105CG' --TOL Gross Adds Registration : DataService - GEO Channel
        
        , 'TB4S001004CS' --TVS CMDU Gross Adds
        , 'TB4S001004CG' --TVS CMDU Gross Adds - GEO Channel
        , 'TB4S001400CS' --TVS Now Gross Adds
        , 'TB4S001400CG' --TVS Now Gross Adds - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-08-13, 13:06:59

DataFrame: 7616 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

C:\Users\Narut4\AppData\Local\Temp\ipykernel_8288\1881320084.py:4: RuntimeWarning: invalid value encountered in cast
  prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)


## Overview(All) by Period

In [4]:
''' Gross Adds(All Channel) Yearly '''

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_yearly_df = ga_yearly_df.loc[ga_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_yearly_df['PRE'] = np.where(ga_yearly_df['METRIC_CD']=='B1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['PRE_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['PRE_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C'] = np.where(ga_yearly_df['METRIC_CD']=='B2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB2S010100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B'] = np.where(ga_yearly_df['METRIC_CD']=='B2S020100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB2S020100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB2S020100CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_REG_CON'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000104CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_CONN'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_CONN(Install)'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102D1CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_REG_DSV'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000105CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_DSV'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000103CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_DSV(Install)'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000103D1CS', ga_yearly_df['P'], 0)
ga_yearly_df['TVS_NOW'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001400CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TVS_CMDU'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001004CS', ga_yearly_df['P'], 0)

# ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_yearly_df_display = ga_yearly_df.copy()
ga_yearly_df_display['ACTUAL_AS_OF'] = ga_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_yearly_df_display[col] = ga_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025,2025-08-13 05:17:31,20250811,"14,721,709","6,967,093","7,755,186","1,357,436","1,041,498","320,044","501,397","366,374","11,532"


In [5]:
''' Gross Adds(All Channel) Quarterly '''

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['METRIC_CD'].isin(v_metric_list)]

ga_quarterly_df['PRE'] = np.where(ga_quarterly_df['METRIC_CD']=='B1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['PRE_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['PRE_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB2S010100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S020100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB2S020100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB2S020100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_REG_CON'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000104CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_CONN'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_CONN(Install)'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102D1CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_REG_DSV'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000105CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_DSV'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000103CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_DSV(Install)'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000103D1CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TVS_NOW'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001400CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TVS_CMDU'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001004CS', ga_quarterly_df['P'], 0)

# ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_quarterly_df_display = ga_quarterly_df.copy()
ga_quarterly_df_display['ACTUAL_AS_OF'] = ga_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_quarterly_df_display[col] = ga_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,20251,2025-08-13 05:17:31,20250331,"6,673,218","3,141,270","3,531,948","569,340","438,816","130,524","198,847","145,692","3,337"
1,20252,2025-08-13 05:17:31,20250630,"5,586,033","2,678,581","2,907,452","550,281","412,045","138,236","209,453","152,388","6,328"
2,20253,2025-08-13 05:17:31,20250811,"2,462,458","1,147,242","1,315,786","237,815","190,637","51,284","93,097","68,294","1,867"


In [6]:
''' Gross Adds(All Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['METRIC_CD'].isin(v_metric_list)]

ga_monthly_df['PRE'] = np.where(ga_monthly_df['METRIC_CD']=='B1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['PRE_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['PRE_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C'] = np.where(ga_monthly_df['METRIC_CD']=='B2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB2S010100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B'] = np.where(ga_monthly_df['METRIC_CD']=='B2S020100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB2S020100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB2S020100CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_REG_CON'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000104CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_CONN'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_CONN(Install)'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102D1CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_REG_DSV'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000105CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_DSV'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000103CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_DSV(Install)'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000103D1CS', ga_monthly_df['P'], 0)
ga_monthly_df['TVS_NOW'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001400CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TVS_CMDU'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001004CS', ga_monthly_df['P'], 0)

# ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_monthly_df_display = ga_monthly_df.copy()
ga_monthly_df_display['ACTUAL_AS_OF'] = ga_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_monthly_df_display[col] = ga_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202501,2025-08-13 05:17:31,20250131,"2,507,456","1,166,199","1,341,257","188,248","142,922","45,326","64,026","46,939",769
1,202502,2025-08-13 05:17:31,20250228,"2,161,914","999,208","1,162,706","183,973","140,856","43,117","62,508","45,846",545
2,202503,2025-08-13 05:17:31,20250331,"2,003,848","975,863","1,027,985","197,119","155,038","42,081","72,313","52,907","2,023"
3,202504,2025-08-13 05:17:31,20250430,"1,879,322","931,089","948,233","197,270","152,200","45,070","66,846","48,482","2,352"
4,202505,2025-08-13 05:17:31,20250531,"1,912,819","912,219","1,000,600","182,723","134,956","47,767","72,984","52,741","2,206"
5,202506,2025-08-13 05:17:31,20250630,"1,793,892","835,273","958,619","170,288","124,889","45,399","69,623","51,165","1,770"
6,202507,2025-08-13 05:17:31,20250731,"1,819,076","842,122","976,954","180,702","140,486","40,216","68,597","50,453","1,553"
7,202508,2025-08-13 05:17:31,20250811,"643,382","305,120","338,832","57,113","50,151","11,068","24,500","17,841",314


In [7]:
''' Gross Adds(All Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['METRIC_CD'].isin(v_metric_list)]

ga_weekly_df['PRE'] = np.where(ga_weekly_df['METRIC_CD']=='B1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['PRE_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['PRE_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C'] = np.where(ga_weekly_df['METRIC_CD']=='B2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB2S010100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B'] = np.where(ga_weekly_df['METRIC_CD']=='B2S020100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB2S020100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB2S020100CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_REG_CON'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000104CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_CONN'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_CONN(Install)'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102D1CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_REG_DSV'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000105CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_DSV'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000103CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_DSV(Install)'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000103D1CS', ga_weekly_df['P'], 0)
ga_weekly_df['TVS_NOW'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001400CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TVS_CMDU'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001004CS', ga_weekly_df['P'], 0)

# ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_weekly_df_display = ga_weekly_df.copy()
ga_weekly_df_display['ACTUAL_AS_OF'] = ga_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_weekly_df_display[col] = ga_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025027,2025-08-13 05:17:31,20250706,"358,890","164,705","194,185","36,748","29,376","7,372","14,699","10,356",273
1,2025028,2025-08-13 05:17:31,20250713,"400,875","184,226","216,649","44,926","35,680","9,246","16,038","11,914",263
2,2025029,2025-08-13 05:17:31,20250720,"416,154","189,305","226,849","45,224","35,082","10,142","15,347","11,303",262
3,2025030,2025-08-13 05:17:31,20250727,"406,128","189,072","217,056","35,039","26,168","8,871","14,496","10,534",427
4,2025031,2025-08-13 05:17:31,20250803,"433,510","207,290","226,220","36,774","29,356","7,418","14,667","11,244",431
5,2025032,2025-08-13 05:17:31,20250810,"446,901","212,074","234,827","39,104","30,869","8,235","15,548","11,382",211
6,2025033,2025-08-13 05:17:31,20250811,0,570,0,0,"4,106",0,"2,302","1,561",0


In [8]:
''' Gross Adds(All Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]

ga_daily_df['PRE'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_T'] = np.where(ga_daily_df['METRIC_CD']=='TB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_D'] = np.where(ga_daily_df['METRIC_CD']=='DB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S010100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_REG_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000104CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_CONN'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_CONN(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102D1CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_REG_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000105CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103D1CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)
# ga_daily_df['TVS_CMDU'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001004CS', ga_daily_df['P'], 0)

# ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_daily_df_display = ga_daily_df.copy()
mod_col_list = ga_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df_display[col] = ga_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202508,20250812,2025-08-13 05:17:31,0,0,0,0,0,0,0,0,0
1,202508,20250811,2025-08-13 05:17:31,0,570,0,0,"4,106",0,"2,302","1,561",0
2,202508,20250810,2025-08-13 05:17:31,"62,493","28,137","34,356","5,063","4,366",697,"1,878","1,566",49
3,202508,20250809,2025-08-13 05:17:31,"64,980","29,758","35,222","5,739","4,699","1,040","2,206","1,678",33
4,202508,20250808,2025-08-13 05:17:31,"70,187","32,713","37,474","5,578","4,447","1,131","2,255","1,548",33
5,202508,20250807,2025-08-13 05:17:31,"63,917","30,419","33,498","5,108","3,998","1,110","2,216","1,618",27
6,202508,20250806,2025-08-13 05:17:31,"61,408","29,240","32,168","5,667","4,309","1,358","2,501","1,602",28
7,202508,20250805,2025-08-13 05:17:31,"60,146","29,231","30,915","6,656","4,940","1,716","2,223","1,679",19
8,202508,20250804,2025-08-13 05:17:31,"63,770","32,576","31,194","5,293","4,110","1,183","2,269","1,691",22
9,202508,20250803,2025-08-13 05:17:31,"62,659","28,557","34,102","6,249","5,339",910,"2,026","1,638",39


## Overview(Geo) by Period

In [9]:
''' Gross Adds(Geo Channel) Yearly '''

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_geo_yearly_df = ga_geo_yearly_df.loc[ga_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_yearly_df['PRE'] = np.where(ga_geo_yearly_df['METRIC_CD']=='B1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['PRE_T'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['PRE_D'] = np.where(ga_geo_yearly_df['METRIC_CD']=='DB1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C'] = np.where(ga_geo_yearly_df['METRIC_CD']=='B2S010100CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C_T'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB2S010100CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C_D'] = np.where(ga_geo_yearly_df['METRIC_CD']=='DB2S010100CG', ga_geo_yearly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_yearly_df['TOL_REG_CON'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000104CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['TOL_CONN'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000102CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_CONN(Install)'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_REG_DSV'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000105CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_DSV'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000103CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_DSV(Install)'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['TVS_NOW'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB4S001400CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TVS_CMDU'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB4S001004CG', ga_geo_yearly_df['P'], 0)

# ga_geo_yearly_df = ga_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_yearly_df = ga_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# ga_geo_yearly_df = ga_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_yearly_df = ga_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_yearly_df = ga_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_geo_yearly_df = ga_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_yearly_df_display = ga_geo_yearly_df.copy()
ga_geo_yearly_df_display['ACTUAL_AS_OF'] = ga_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_yearly_df_display[col] = ga_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025,2025-08-13 05:17:31,20250811,"12,472,783","6,943,643","5,529,708","606,638","510,329","98,031","382,772","283,483","11,510"


In [10]:
''' Gross Adds(Geo Channel) Quarterly '''

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_geo_quarterly_df = ga_geo_quarterly_df.loc[ga_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_quarterly_df['PRE'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='B1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['PRE_T'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['PRE_D'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='DB1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='B2S010100CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C_T'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB2S010100CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C_D'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='DB2S010100CG', ga_geo_quarterly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_quarterly_df['TOL_REG_CON'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000104CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['TOL_CONN'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000102CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_CONN(Install)'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_REG_DSV'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000105CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_DSV'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000103CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_DSV(Install)'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['TVS_NOW'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB4S001400CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TVS_CMDU'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB4S001004CG', ga_geo_quarterly_df['P'], 0)

# ga_geo_quarterly_df = ga_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_quarterly_df = ga_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# ga_geo_quarterly_df = ga_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_quarterly_df = ga_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_quarterly_df = ga_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_geo_quarterly_df = ga_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_quarterly_df_display = ga_geo_quarterly_df.copy()
ga_geo_quarterly_df_display['ACTUAL_AS_OF'] = ga_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_quarterly_df_display[col] = ga_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,20251,2025-08-13 05:17:31,20250331,"5,553,809","3,132,044","2,421,765","262,488","223,283","39,205","150,337","111,869","3,329"
1,20252,2025-08-13 05:17:31,20250630,"4,835,811","2,667,703","2,168,108","240,824","200,548","40,276","161,808","119,281","6,314"
2,20253,2025-08-13 05:17:31,20250811,"2,083,163","1,143,896","939,835","103,326","86,498","18,550","70,627","52,333","1,867"


In [11]:
''' Gross Adds(Geo Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
ga_geo_monthly_df = ga_geo_monthly_df.loc[ga_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_monthly_df['PRE'] = np.where(ga_geo_monthly_df['METRIC_CD']=='B1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['PRE_T'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['PRE_D'] = np.where(ga_geo_monthly_df['METRIC_CD']=='DB1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C'] = np.where(ga_geo_monthly_df['METRIC_CD']=='B2S010100CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C_T'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB2S010100CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C_D'] = np.where(ga_geo_monthly_df['METRIC_CD']=='DB2S010100CG', ga_geo_monthly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_monthly_df['TOL_REG_CON'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000104CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['TOL_CONN'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000102CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_CONN(Install)'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_REG_DSV'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000105CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_DSV'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000103CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_DSV(Install)'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['TVS_NOW'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB4S001400CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TVS_CMDU'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB4S001004CG', ga_geo_monthly_df['P'], 0)

# ga_geo_monthly_df = ga_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_monthly_df = ga_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# ga_geo_monthly_df = ga_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_monthly_df = ga_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_monthly_df = ga_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_geo_monthly_df = ga_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_monthly_df_display = ga_geo_monthly_df.copy()
ga_geo_monthly_df_display['ACTUAL_AS_OF'] = ga_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_monthly_df_display[col] = ga_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202501,2025-08-13 05:17:31,20250131,"2,038,766","1,163,218","875,548","83,267","70,852","12,415","46,010","34,626",765
1,202502,2025-08-13 05:17:31,20250228,"1,798,194","996,201","801,993","83,956","71,425","12,531","48,150","35,785",543
2,202503,2025-08-13 05:17:31,20250331,"1,716,849","972,625","744,224","95,265","81,006","14,259","56,177","41,458","2,021"
3,202504,2025-08-13 05:17:31,20250430,"1,636,131","926,446","709,685","92,697","78,554","14,143","51,632","37,924","2,349"
4,202505,2025-08-13 05:17:31,20250531,"1,662,837","908,569","754,268","78,554","64,712","13,842","56,167","41,428","2,196"
5,202506,2025-08-13 05:17:31,20250630,"1,536,843","832,688","704,155","69,573","57,282","12,291","54,009","39,929","1,769"
6,202507,2025-08-13 05:17:31,20250731,"1,540,810","839,672","701,138","78,858","64,962","13,896","52,221","38,873","1,553"
7,202508,2025-08-13 05:17:31,20250811,"542,353","304,224","238,697","24,468","21,536","4,654","18,406","13,460",314


In [12]:
''' Gross Adds(Geo Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_geo_weekly_df = ga_geo_weekly_df.loc[ga_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_weekly_df['PRE'] = np.where(ga_geo_weekly_df['METRIC_CD']=='B1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['PRE_T'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['PRE_D'] = np.where(ga_geo_weekly_df['METRIC_CD']=='DB1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C'] = np.where(ga_geo_weekly_df['METRIC_CD']=='B2S010100CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C_T'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB2S010100CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C_D'] = np.where(ga_geo_weekly_df['METRIC_CD']=='DB2S010100CG', ga_geo_weekly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_weekly_df['TOL_REG_CON'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000104CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['TOL_CONN'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000102CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_CONN(Install)'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_REG_DSV'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000105CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_DSV'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000103CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_DSV(Install)'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['TVS_NOW'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB4S001400CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TVS_CMDU'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB4S001004CG', ga_geo_weekly_df['P'], 0)

# ga_geo_weekly_df = ga_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_weekly_df = ga_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# ga_geo_weekly_df = ga_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_weekly_df = ga_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_weekly_df = ga_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_geo_weekly_df = ga_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_weekly_df_display = ga_geo_weekly_df.copy()
ga_geo_weekly_df_display['ACTUAL_AS_OF'] = ga_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_weekly_df_display[col] = ga_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025027,2025-08-13 05:17:31,20250706,"307,362","164,256","143,106","16,164","13,184","2,980","11,298","8,087",273
1,2025028,2025-08-13 05:17:31,20250713,"340,253","183,578","156,675","18,737","15,228","3,509","12,481","9,342",263
2,2025029,2025-08-13 05:17:31,20250720,"351,232","188,742","162,490","20,219","16,717","3,502","11,337","8,557",262
3,2025030,2025-08-13 05:17:31,20250727,"339,937","188,545","151,392","15,097","12,632","2,465","10,988","7,972",427
4,2025031,2025-08-13 05:17:31,20250803,"366,741","206,780","159,961","16,386","13,378","3,008","11,190","8,588",431
5,2025032,2025-08-13 05:17:31,20250810,"377,638","211,427","166,211","16,723","13,637","3,086","11,491","8,610",211
6,2025033,2025-08-13 05:17:31,20250811,0,568,0,0,"1,722",0,"1,842","1,177",0


In [13]:
''' Gross Adds(Geo Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_geo_daily_df = ga_geo_daily_df.loc[ga_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_daily_df['PRE'] = np.where(ga_geo_daily_df['METRIC_CD']=='B1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C'] = np.where(ga_geo_daily_df['METRIC_CD']=='B2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB2S010100CG', ga_geo_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_daily_df['TOL_REG_CON'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000104CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TOL_CONN'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_CONN(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_REG_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000105CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TVS_NOW'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001400CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TVS_CMDU'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001004CG', ga_geo_daily_df['P'], 0)

# ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_daily_df_display = ga_geo_daily_df.copy()
mod_col_list = ga_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_daily_df_display[col] = ga_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202508,20250812,2025-08-13 05:17:31,0,0,0,0,0,0,0,0,0
1,202508,20250811,2025-08-13 05:17:31,0,568,0,0,"1,722",0,"1,842","1,177",0
2,202508,20250810,2025-08-13 05:17:31,"52,621","28,052","24,569","2,470","1,976",494,"1,433","1,110",49
3,202508,20250809,2025-08-13 05:17:31,"53,323","29,655","23,668","2,445","2,017",428,"1,693","1,174",33
4,202508,20250808,2025-08-13 05:17:31,"59,084","32,600","26,484","2,239","1,847",392,"1,644","1,193",33
5,202508,20250807,2025-08-13 05:17:31,"54,065","30,328","23,737","2,142","1,765",377,"1,626","1,214",27
6,202508,20250806,2025-08-13 05:17:31,"52,332","29,156","23,176","2,293","1,886",407,"1,755","1,245",28
7,202508,20250805,2025-08-13 05:17:31,"51,598","29,151","22,447","2,681","2,172",509,"1,632","1,335",19
8,202508,20250804,2025-08-13 05:17:31,"54,615","32,485","22,130","2,453","1,974",479,"1,708","1,339",22
9,202508,20250803,2025-08-13 05:17:31,"52,532","28,491","24,041","2,791","2,184",607,"1,550","1,162",39


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202508


### Prepaid

In [16]:
''' Prepaid (All Channel) '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[~prepaid_df['METRIC_NAME'].str.contains('GEO')]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Prepaid,B1S000101CS,Prepaid Gross Adds,2025-08-13 05:17:31,"643,382","643,382","566,382","566,382","557,641"
1,202508,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"338,832","338,832","333,297","333,297","328,989"
2,202508,Prepaid,TB1S000101CS,Prepaid Gross Adds : TMH,2025-08-13 05:17:31,"305,120","305,120","233,239","233,239","228,805"


In [17]:
''' Prepaid (Geo Channel) '''

v_product_grp = 'Prepaid'

prepaid_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['METRIC_NAME'].str.contains('GEO')]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_geo_df = prepaid_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Prepaid,B1S000101CG,Prepaid Gross Adds - GEO Channel,2025-08-13 05:17:31,0,"542,353","469,453","469,453","461,528"
1,202508,Prepaid,DB1S000101CG,Prepaid Gross Adds : DTAC - GEO Channel,2025-08-13 05:17:31,0,"238,697","236,435","236,435","232,943"
2,202508,Prepaid,TB1S000101CG,Prepaid Gross Adds : TMH - GEO Channel,2025-08-13 05:17:31,0,"304,224","233,172","233,172","228,738"


### Postpaid

In [18]:
''' Postpaid B2C (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[~postpaid_b2c_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Postpaid,B2S010100CS,Postpaid Gross Adds B2C,2025-08-13 05:17:31,"57,113","57,113","39,069","38,961","38,825"
1,202508,Postpaid,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-08-13 05:17:31,"11,068","11,068","10,989","10,989","10,989"
2,202508,Postpaid,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-08-13 05:17:31,"50,151","50,151","30,497","30,497","30,497"


In [19]:
''' Postpaid B2C (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Postpaid,B2S010100CG,Postpaid Gross Adds B2C - GEO Channel,2025-08-13 05:17:31,0,"24,468","24,463","24,006","23,337"
1,202508,Postpaid,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,2025-08-13 05:17:31,0,"4,654","4,649","4,649","4,649"
2,202508,Postpaid,TB2S010100CG,Postpaid Gross Adds B2C : TMH - GEO Channel,2025-08-13 05:17:31,0,"21,536","21,532","21,532","21,532"


In [20]:
''' Postpaid B2B (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[~postpaid_b2b_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Postpaid,B2S020100CS,Postpaid Gross Adds B2B,2025-08-13 05:17:31,"9,655","9,655","2,923","2,517","2,442"
1,202508,Postpaid,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-08-13 05:17:31,"3,834","3,834","1,475","1,475","1,475"
2,202508,Postpaid,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-08-13 05:17:31,"6,537","6,537","1,695","1,696","1,696"


In [21]:
''' Postpaid B2B (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,Postpaid,B2S020100CG,Postpaid Gross Adds B2B - GEO Channel,2025-08-13 05:17:31,0,"1,885","1,777","1,286","1,411"
1,202508,Postpaid,DB2S020100CG,Postpaid Gross Adds B2B : DTAC - GEO Channel,2025-08-13 05:17:31,0,396,396,396,396
2,202508,Postpaid,TB2S020100CG,Postpaid Gross Adds B2B : TMH - GEO Channel,2025-08-13 05:17:31,0,"1,695","1,695","1,696","1,696"


### TOL

In [22]:
''' TOL Registration : Consumer (All & Geo) '''

v_product_grp = 'TOL'

tol_reg_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['METRIC_NAME'].str.contains('Registration')]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['METRIC_NAME'].str.contains('Consumer')]
# tol_reg_con_df = tol_reg_con_df.loc[~tol_reg_con_df['METRIC_NAME'].str.contains('GEO')]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_reg_con_df = tol_reg_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_reg_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-08-13 05:17:31,"24,500","24,500","18,411","18,411","18,143"
1,202508,TOL,TB3S000104CG,TOL Gross Adds Registration : Consumer - GEO C...,2025-08-13 05:17:31,0,"18,406","18,404","18,404","18,136"


In [23]:
''' TOL Registration : DataService (All & Geo) '''

v_product_grp = 'TOL'

tol_reg_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['METRIC_NAME'].str.contains('Registration')]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['METRIC_NAME'].str.contains('DataService')]
# tol_reg_dsv_df = tol_reg_dsv_df.loc[~tol_reg_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_reg_dsv_df = tol_reg_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_reg_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-08-13 05:17:31,568,568,24,24,24
1,202508,TOL,TB3S000105CG,TOL Gross Adds Registration : DataService - GE...,2025-08-13 05:17:31,0,24,24,24,24


In [24]:
''' TOL Connected : Consumer (All Channel) '''

v_product_grp = 'TOL'

tol_conn_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_df = tol_conn_con_df.loc[~tol_conn_con_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_df = tol_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"17,841","17,841","13,460","13,460","13,265"
1,202508,TOL,TB3S000102D1CS,TOL Gross Adds Connected : Consumer (Install L...,2025-08-13 05:17:31,"17,841","17,841","17,841","17,841","17,841"


In [25]:
''' TOL Connected : Consumer (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_con_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_geo_df = tol_conn_con_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3S000102CG,TOL Gross Adds Connected : Consumer - GEO Channel,2025-08-13 05:17:31,0,"13,460","13,460","13,460","13,265"
1,202508,TOL,TB3S000102D1CG,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-08-13 05:17:31,0,"13,460","13,460","13,460","13,460"


In [26]:
''' TOL Connected : DataService (All Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[~tol_conn_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_df = tol_conn_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-08-13 05:17:31,420,420,10,10,10
1,202508,TOL,TB3S000103D1CS,TOL Gross Adds Connected : DataService (Instal...,2025-08-13 05:17:31,420,420,10,10,10


In [27]:
''' TOL Connected : DataService (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[~tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-08-13 05:17:31,420,420,10,10,10
1,202508,TOL,TB3S000103D1CS,TOL Gross Adds Connected : DataService (Instal...,2025-08-13 05:17:31,420,420,10,10,10


### TVS

In [28]:
''' TVS Now '''

v_product_grp = 'TVS'

tvs_now_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_now_df = tvs_now_df.loc[tvs_now_df['METRIC_NAME'].str.contains('Now')]
tvs_now_df = tvs_now_df.loc[tvs_now_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_now_df = tvs_now_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_now_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TVS,TB4S001400CS,TVS Now Gross Adds,2025-08-13 05:17:31,314,314,314,314,314
1,202508,TVS,TB4S001400CG,TVS Now Gross Adds - GEO Channel,2025-08-13 05:17:31,0,314,314,314,314


In [29]:
''' TVS CMDU '''

v_product_grp = 'TVS'

tvs_cmdu_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['METRIC_NAME'].str.contains('CMDU')]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_cmdu_df = tvs_cmdu_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_cmdu_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202508,TVS,TB4S001004CS,TVS CMDU Gross Adds,2025-08-13 05:17:31,11,11,11,11,11
1,202508,TVS,TB4S001004CG,TVS CMDU Gross Adds - GEO Channel,2025-08-13 05:17:31,0,0,0,0,0


## ** Issue History

In [30]:
''' Monthly : TOL Gross Adds Connected : Consumer '''

v_metric_cd = 'TB3S000102CS'

issue_tol_ga_conn_con_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_tol_ga_conn_con_df = issue_tol_ga_conn_con_df.loc[issue_tol_ga_conn_con_df['TM_KEY_MTH']>=202401]
issue_tol_ga_conn_con_df = issue_tol_ga_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_tol_ga_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202501,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"46,939","46,939","34,630","34,630","36,115"
1,202502,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"45,846","45,846","35,794","35,319","35,506"
2,202503,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"52,907","52,907","41,460","41,460","41,080"
3,202504,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"48,482","48,482","37,922","37,883","37,550"
4,202505,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"52,741","52,741","41,427","41,427","41,039"
5,202506,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"51,165","51,165","39,927","39,927","39,514"
6,202507,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"50,453","50,453","38,874","38,874","38,480"
7,202508,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"17,841","17,841","13,460","13,460","13,265"


In [31]:
''' Daily : TOL Gross Adds Connected : Consumer '''

v_metict_cd = 'TB3S000102CS'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_tol_conn_con_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_tol_conn_con_df = issue_daily_tol_conn_con_df.loc[issue_daily_tol_conn_con_df['METRIC_CD']==v_metict_cd]
issue_daily_tol_conn_con_df = issue_daily_tol_conn_con_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_tol_conn_con_df_display = issue_daily_tol_conn_con_df.copy()
mod_col_list = issue_daily_tol_conn_con_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_tol_conn_con_df_display[col] = issue_daily_tol_conn_con_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_tol_conn_con_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202508,20250812,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,nan,nan,nan,nan
1,202508,20250811,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,561","1,177","1,177","1,167"
2,202508,20250810,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,566","1,110","1,110","1,088"
3,202508,20250809,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,678","1,174","1,174","1,157"
4,202508,20250808,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,548","1,193","1,193","1,172"
5,202508,20250807,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,618","1,214","1,214","1,188"
6,202508,20250806,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,602","1,245","1,245","1,233"
7,202508,20250805,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,679","1,335","1,335","1,319"
8,202508,20250804,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,691","1,339","1,339","1,313"
9,202508,20250803,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-08-13 05:17:31,"1,638","1,162","1,162","1,144"


## ** Current Issue

In [32]:
''' Daily : Prepaid Gross Adds : DTAC '''

v_metict_cd = 'DB1S000101CS'
# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

issue_daily_pre_dtac_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_pre_dtac_df = issue_daily_pre_dtac_df.loc[issue_daily_pre_dtac_df['METRIC_CD']==v_metict_cd]
issue_daily_pre_dtac_df = issue_daily_pre_dtac_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_pre_dtac_df_display = issue_daily_pre_dtac_df.copy()
mod_col_list = issue_daily_pre_dtac_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_pre_dtac_df_display[col] = issue_daily_pre_dtac_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_pre_dtac_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202508,20250812,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,nan,nan,nan,nan
1,202508,20250811,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,nan,nan,nan,nan
2,202508,20250810,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"34,356","32,837","32,837","32,470"
3,202508,20250809,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"35,222","34,617","34,617","34,217"
4,202508,20250808,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"37,474","36,916","36,916","36,458"
5,202508,20250807,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"33,498","32,913","32,913","32,477"
6,202508,20250806,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"32,168","31,917","31,917","31,491"
7,202508,20250805,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"30,915","30,481","30,481","30,095"
8,202508,20250804,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"31,194","30,835","30,835","30,447"
9,202508,20250803,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-08-13 05:17:31,"34,102","33,637","33,637","33,162"
